In [1]:
import pandas as pd
import numpy as np

In [2]:
edu_set = pd.read_excel("C:\OceanCleanup\State_Education.xlsx")
pop_set = pd.read_excel("C:\OceanCleanup\State_Population.xlsx", header=1)
county_edu_set = pd.read_excel(r'C:\OceanCleanup\Education.xls',header=4)
county_eco_set = pd.read_excel(r"C:\OceanCleanup\Unemployment.xls")
county_pop_set = pd.read_excel(r"C:\OceanCleanup\PopulationEstimates.xls")
visit_set = pd.read_excel(r"C:\OceanCleanup\US States Visited 2018.xlsx",header=6)
state_abbre = pd.read_excel(r"C:\OceanCleanup\State_abbreviations.xlsx")
state_coastline = pd.read_excel("C:\OceanCleanup\State_Coastline.xlsx")

state_coastline.State = state_coastline.State.apply(lambda x: x.replace(u'\xa0', u''))
state_coastline.Coastline = state_coastline.Coastline.apply(lambda x: x.split("mi")[0])
state_coastline.Coastline = state_coastline.Coastline.apply(lambda x: x.replace(u'\xa0', u''))
state_coastline.Coastline = state_coastline.Coastline.apply(lambda x: x.replace(u',', u''))
state_coastline.Coastline = state_coastline.Coastline.astype("int")

state_abbre.FullName = state_abbre.FullName.apply(lambda x: x.replace(u'\xa0', u''))

"""
edu_set.State = edu_set.State.apply(lambda x: x.split("\xa0")[1])
pop_set.State = pop_set.State.apply(lambda x: x.split(".")[1])
"""
#state_set = pop_set.join(edu_set.set_index("State"), on="State")

'\nedu_set.State = edu_set.State.apply(lambda x: x.split("\xa0")[1])\npop_set.State = pop_set.State.apply(lambda x: x.split(".")[1])\n'

In [3]:
visit_set = visit_set.rename(columns={"STATE/TERRITORY":"State","MARKET":"MarketShare"})
#visit_set = visit_set.set_index("State")
visit_set = visit_set.drop([2018, 'VISITATION', '%', 'MARKET.1', 'VISITATION.1'],axis=1)

county_edu_set = county_edu_set.rename(columns={"Area name":"County","Percent of adults with a bachelor's degree or higher, 2014-18":"BachelorOrHigher%","FIPS Code":"FIPS"})
county_edu_set = county_edu_set[["FIPS","State","County","BachelorOrHigher%"]]
#county_edu_set = county_edu_set.set_index("State")

county_eco_set.Area_name = county_eco_set.Area_name.apply(lambda x: x.split(",")[0])
county_eco_set = county_eco_set[["FIPS","State","Area_name","Median_Household_Income_2018"]]
#county_eco_set = county_eco_set.set_index("County")
county_eco_set = county_eco_set.rename(columns={"Median_Household_Income_2018":"medHousInc","Area_name":"County"})

county_pop_set.Area_Name = county_pop_set.Area_Name.apply(lambda x: x.split(",")[0])
county_pop_set = county_pop_set[["FIPS","State","Area_Name","POP_ESTIMATE_2018"]]
county_pop_set = county_pop_set.rename(columns={"POP_ESTIMATE_2018":"Population","Area_Name":"County"})

temp = county_pop_set.merge(county_eco_set,on="FIPS").merge(county_edu_set,on="FIPS")
temp = temp[["FIPS","State","County_y","Population","medHousInc","BachelorOrHigher%"]]
temp = temp.rename(columns={"County_y":"County","BachelorOrHigher%":"Bachelor"})
temp = temp.merge(visit_set,on="State")
temp = temp.merge(state_abbre, on="State")
temp = temp.merge(state_coastline,how="left",left_on = "FullName", right_on="State")
temp.Coastline = temp.Coastline.fillna(0)
temp = temp.drop(columns="State_y")
temp = temp.rename(columns={"State_x":"State"})

temp['Sea'] = pd.Series(~np.isnan(temp.Coastline))

temp.to_excel("County_data.xlsx")

In [4]:
cleanup_set = pd.read_excel("C:\OceanCleanup\Data_Level5_BAH_OceanCleanup.xlsx")

print(cleanup_set.shape[0])
#cleanup_set.dropna(subset=['State',"Total Items Collected"], how ="any")
cleanup_set.dropna()
print(cleanup_set.shape[0])


cleanup_set["Year"] = pd.DatetimeIndex(cleanup_set["Cleanup Date"]).year
cleanup_set.dropna(subset=["Year"], how ="any")
cleanup_set.Year.astype("str")
cleanup_set["Month"] = pd.DatetimeIndex(cleanup_set["Cleanup Date"]).month
cleanup_set.dropna(subset=["Month"], how ="any")
cleanup_set.Month.astype("str")
cleanup_set["Year Month"] = pd.to_datetime(cleanup_set["Cleanup Date"]).dt.to_period('M')


cleanup_set.State = cleanup_set.State.astype('str')
cleanup_set.State = cleanup_set.State.apply(lambda x: x.split(",")[0])

cleanup_set.Zone = cleanup_set.Zone.astype('str')
cleanup_set.Zone = cleanup_set.Zone.apply(lambda x: x.split(",")[0])
cleanup_set = cleanup_set.rename(columns={"Zone":"County","Total Items Collected":"totalItems"})


cleanup_set[["Latitude","Longitude"]] = cleanup_set.GPS.str.split(",",expand = True)
#pd.unique(cleanup_set.State)

"""
cleanup = cleanup_set.join(state_set.set_index("State"), on="State",how="left")
cleanup = cleanup[['State','Country','County','Latitude',
                   'Longitude','Cleanup Type','Cleanup Date',
                   'Group Name','Adults','Children',
                   'People','Pounds','Miles',
                   'Total Items Collected','Month',
                   'Year','Year Month']]
"""

#cleanup.to_excel("cleaned_data.xlsx")

37903
37903


'\ncleanup = cleanup_set.join(state_set.set_index("State"), on="State",how="left")\ncleanup = cleanup[[\'State\',\'Country\',\'County\',\'Latitude\',\n                   \'Longitude\',\'Cleanup Type\',\'Cleanup Date\',\n                   \'Group Name\',\'Adults\',\'Children\',\n                   \'People\',\'Pounds\',\'Miles\',\n                   \'Total Items Collected\',\'Month\',\n                   \'Year\',\'Year Month\']]\n'

In [5]:
cleanup_total = cleanup_set[['County',"State",'totalItems',"People"]].groupby(['County',"State"]).sum()
data = cleanup_total.merge(temp,how="left",left_on=["County","State"],right_on=["County","FullName"])
data = data.drop(columns="FIPS")
print(cleanup_total.shape)
print(data.shape)
data = data.dropna(how="any",subset=['County',"State",'totalItems',"People"])
print(data.shape)
data.to_excel("Clean_Data.xlsx",index=False)
data.to_csv(r"C:\OceanCleanup\Clean_Data.csv",index=False)

(950, 2)
(983, 11)
(781, 11)
